<a href="https://colab.research.google.com/github/Dhaneshkp/DesktopAssistant/blob/main/DATA_processing_conv1D_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

Everst_data=pd.read_csv('average_premium.csv')
data=Everst_data
data['Attained_age']=data['Age']+data['Duration']
data['Premium_rate']=data['BasePremiumAmount']/data['BaseNARAmount']*10000
#exclude_cols = ['Gender','RiskClass']
Coulmns_to_keep=['Age','Premium_rate','Gender','RiskClass','Duration']
#data=data[data['SingleOrJointType']=="Single"]
#data=data[data['Gender']=="Male"]
#data=data[data['RiskClass']=="NS1"]
data=data[Coulmns_to_keep]
data=data[(data['RiskClass']!="SS1") & (data['RiskClass']!="SS2")& (data['RiskClass']!="SP1")]
data['index']=data.index
numerical_columns = data.select_dtypes(include=['number']).columns
categorical_columns = data.select_dtypes(include=['object']).columns
data_copy=data.head(8657*5).copy()
data=data.head(8657*5)

<ipython-input-1-90565239e93f>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['index']=data.index


In [2]:
# Create a multi-index dataframe using Age, Duration, and a combined Gender and RiskClass column

import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
data['RiskClass'] = data['RiskClass'].replace({'N+1': 'NP2', 'NP1': 'NP2'}).replace({'NS3': 'NS2', 'NS2': 'NS2', 'NR1': 'NS2', 'NS1' : 'NS2'})
print(data)
print(data['RiskClass'].unique())
multi_index_df = data.set_index(['Age', 'Duration', data['Gender'] + '_' + data['RiskClass']])

# Convert the multi-index dataframe to a regular dataframe
result_df = multi_index_df.reset_index().drop(['Gender', 'RiskClass'], axis=1)
print(result_df)
result_df['level_2'].unique()

       Age  Premium_rate  Gender RiskClass  Duration  index
0       52     32.884713  Female       NP2         8      0
1       25      7.822222    Male       NP2         7      1
2       20      3.640212  Female       NS2         8      2
4       38     11.758730    Male       NP2         8      4
5       30      5.480635    Male       NS2         8      5
...    ...           ...     ...       ...       ...    ...
47373   23      6.720635    Male       NS2         3  47373
47374   29      3.841270  Female       NS2         4  47374
47375   44     11.541446    Male       NP2         4  47375
47376   36      6.016629    Male       NS2         4  47376
47378   33      4.118519  Female       NS2         4  47378

[43285 rows x 6 columns]
['NP2' 'NS2']
       Age  Duration     level_2  Premium_rate  index
0       52         8  Female_NP2     32.884713      0
1       25         7    Male_NP2      7.822222      1
2       20         8  Female_NS2      3.640212      2
3       38         8    

array(['Female_NP2', 'Male_NP2', 'Female_NS2', 'Male_NS2'], dtype=object)

In [3]:
import numpy as np
import pandas as pd

# Before removing NAs
print("Before removing NAs:")
print(result_df.describe(include='all'))
print(result_df.count())

# Drop rows with missing values
result_df_no_na = result_df.dropna()

# After removing NAs
print("\nAfter removing NAs:")
print(result_df_no_na.describe(include='all'))
print(result_df_no_na.count())


Before removing NAs:
                 Age      Duration     level_2  Premium_rate         index
count   43285.000000  43285.000000       43285  43285.000000  43285.000000
unique           NaN           NaN           4           NaN           NaN
top              NaN           NaN  Female_NP2           NaN           NaN
freq             NaN           NaN       13545           NaN           NaN
mean       37.263994      4.845535         NaN     13.024018  23686.246436
std        11.245521      2.555028         NaN     22.319405  13685.301612
min        18.000000      1.000000         NaN     -9.301587      0.000000
25%        28.000000      3.000000         NaN      4.137566  11843.000000
50%        36.000000      5.000000         NaN      7.228571  23676.000000
75%        45.000000      7.000000         NaN     13.192240  35539.000000
max        70.000000     10.000000         NaN   1105.705706  47378.000000
Age             43285
Duration        43285
level_2         43285
Premium_rate 

In [4]:
# prompt: for teh data ccreate the difference of value of premium rate with the avearege in each frame where frameis oof length  age 5 and duration lengthe of 3

import numpy as np
import pandas as pd

# Assuming 'result_df_no_na' is your DataFrame from the preceding code

# Define frame sizes
age_frame_size = 5
duration_frame_size = 3

# Create a new column for the difference from average premium rate
result_df_no_na['Premium_Diff'] = 0

# Iterate over age and duration frames
for age_start in range(result_df_no_na['Age'].min(), result_df_no_na['Age'].max() + 1, age_frame_size):
  for duration_start in range(result_df_no_na['Duration'].min(), result_df_no_na['Duration'].max() + 1, duration_frame_size):
    age_end = age_start + age_frame_size
    duration_end = duration_start + duration_frame_size

    # Filter data for the current frame
    frame_data = result_df_no_na[
        (result_df_no_na['Age'] >= age_start) & (result_df_no_na['Age'] < age_end) &
        (result_df_no_na['Duration'] >= duration_start) & (result_df_no_na['Duration'] < duration_end)
    ]

    # Calculate the average premium rate for the frame
    avg_premium = round(frame_data['Premium_rate'].mean(),2)

    # Calculate the difference from average for each row in the frame
    result_df_no_na.loc[frame_data.index, 'Premium_Diff'] = round(frame_data['Premium_rate'] - avg_premium,2)

print(result_df_no_na.sort_values(by =['level_2','Age','Duration']))


       Age  Duration     level_2  Premium_rate  index  Premium_Diff
1245    18         1  Female_NP2      1.950265   1360         -2.35
8302    18         1  Female_NP2      1.950265   9088         -2.35
15882   18         1  Female_NP2      1.969550  17368         -2.33
27857   18         1  Female_NP2      2.140822  30482         -2.16
32246   18         1  Female_NP2      1.790476  35304         -2.51
...    ...       ...         ...           ...    ...           ...
27284   70         4    Male_NS2    149.312169  29857         19.08
34612   70         4    Male_NS2    149.320635  37905         19.09
19921   70         7    Male_NS2    237.448133  21778         17.19
22421   70        10    Male_NS2    318.370370  24521         50.38
37180   70        10    Male_NS2    318.364444  40714         50.37

[43285 rows x 6 columns]


In [5]:
# Calculate average Premium_Diff for duplicates
average_premium_diff = result_df_no_na.groupby(['Age', 'Duration', 'level_2'])['Premium_Diff'].mean().reset_index()

average_premium_diff.pivot(index=['level_2','Age'], columns='Duration', values='Premium_Diff')
average_premium_diff.to_csv('average_premium_diff.csv')

In [6]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, Flatten, Dense, MaxPooling1D

In [7]:
# prompt: for the dsata  result_df_no_na one hot encode the level_2

import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# Assuming 'result_df_no_na' is your DataFrame from the preceding code

# Create a OneHotEncoder object
encoder = OneHotEncoder(handle_unknown='ignore')

# Fit the encoder to the 'level_2' column and transform it
encoded_data = encoder.fit_transform(result_df_no_na[['level_2']])

# Create a DataFrame from the encoded data
encoded_df = pd.DataFrame(encoded_data.toarray(), columns=encoder.get_feature_names_out(['level_2']))

# Concatenate the encoded DataFrame with the original DataFrame
result_df_encoded = pd.concat([result_df_no_na, encoded_df], axis=1)

# Optionally, drop the original 'level_2' column
result_df_encoded = result_df_encoded.drop('level_2', axis=1)

result_df_encoded.drop(['index','Premium_Diff'], axis=1,inplace=True)
result_df_encoded

,Age,Duration,Premium_rate,level_2_Female_NP2,level_2_Female_NS2,level_2_Male_NP2,level_2_Male_NS2
0,52,8,32.884713,1.0,0.0,0.0,0.0
1,25,7,7.822222,0.0,0.0,1.0,0.0
2,20,8,3.640212,0.0,1.0,0.0,0.0
3,38,8,11.758730,0.0,0.0,1.0,0.0
4,30,8,5.480635,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...
43280,23,3,6.720635,0.0,0.0,0.0,1.0
43281,29,4,3.841270,0.0,1.0,0.0,0.0
43282,44,4,11.541446,0.0,0.0,1.0,0.0
43283,36,4,6.016629,0.0,0.0,0.0,1.0


In [8]:
#test_df_with_anomalies = result_df_encoded.iloc[X_test.index].copy()  # X_test is a numpy array and does not have an index
# Reshape the data to 3D for CNN input
data_array = result_df_encoded.values  # Extract the underlying NumPy array
data_reshaped = data_array.reshape(-1, 7, 1)

# Split the data into training and test sets
X_train, X_test = train_test_split(data_reshaped, test_size=0.2, random_state=42)
train_indices, test_indices = train_test_split(result_df_encoded.index, test_size=0.2, random_state=42)
# Define the CNN architecture
input_shape = (X_train.shape[1], X_train.shape[2])
input_layer = Input(shape=input_shape)

conv_layer = Conv1D(32, kernel_size=3, activation="relu")(input_layer)
conv_layer = MaxPooling1D(pool_size=1)(conv_layer)
conv_layer = Flatten()(conv_layer)

dense_layer = Dense(10, activation="relu")(conv_layer)
output_layer = Dense(7, activation="linear")(dense_layer)

cnn_model = Model(inputs=input_layer, outputs=output_layer)
cnn_model.compile(optimizer='adam', loss='mse')

# Train the model
history = cnn_model.fit(X_train, X_train,
                        epochs=50,
                        batch_size=32,
                        validation_split=0.1,
                        verbose=1)

# Predict the training data
pred_train = cnn_model.predict(X_train)
mse_train = np.mean(np.power(X_train.reshape(-1, 7) - pred_train, 2), axis=1)

# Predict the test data
pred_test = cnn_model.predict(X_test)
mse_test = np.mean(np.power(X_test.reshape(-1, 7) - pred_test, 2), axis=1)

# Define a threshold for anomaly detection
threshold = np.percentile(mse_train, 95)

# Identify anomalies in the test set
anomalies = mse_test > threshold

# Print the results
print(f"Threshold: {threshold}")
print(f"Number of anomalies in the test set: {np.sum(anomalies)}")


Epoch 1/50
974/974 [==============================] - 10s 8ms/step - loss: 42.8845 - val_loss: 0.6552
Epoch 2/50
974/974 [==============================] - 2s 2ms/step - loss: 0.4399 - val_loss: 0.3295
Epoch 3/50
974/974 [==============================] - 2s 2ms/step - loss: 0.2420 - val_loss: 0.1858
Epoch 4/50
974/974 [==============================] - 2s 2ms/step - loss: 0.1384 - val_loss: 0.1090
Epoch 5/50
974/974 [==============================] - 2s 2ms/step - loss: 0.1140 - val_loss: 0.0974
Epoch 6/50
974/974 [==============================] - 3s 3ms/step - loss: 0.0925 - val_loss: 0.0856
Epoch 7/50
974/974 [==============================] - 2s 2ms/step - loss: 0.1030 - val_loss: 0.0797
Epoch 8/50
974/974 [==============================] - 2s 2ms/step - loss: 0.0815 - val_loss: 0.0752
Epoch 9/50
974/974 [==============================] - 2s 2ms/step - loss: 0.0797 - val_loss: 0.0755
Epoch 10/50
974/974 [==============================] - 2s 2ms/step - loss: 0.1131 - val_loss: 0.07

In [9]:
# Create a copy of the original test DataFrame
test_df_with_anomalies = result_df_encoded.iloc[test_indices].copy()

# Add an 'anomaly' column to the test DataFrame
test_df_with_anomalies['anomaly'] = anomalies

print(f"Threshold: {threshold}")
print(f"Number of anomalies in the test set: {np.sum(anomalies)}")

# Display the test DataFrame with anomaly tags
test_df_with_anomalies

Threshold: 0.1461205546688863
Number of anomalies in the test set: 427


,Age,Duration,Premium_rate,level_2_Female_NP2,level_2_Female_NS2,level_2_Male_NP2,level_2_Male_NS2,anomaly
42657,19,3,3.182540,1.0,0.0,0.0,0.0,False
38367,30,5,4.298786,0.0,0.0,1.0,0.0,False
17831,40,3,8.730159,0.0,0.0,1.0,0.0,True
7236,45,3,10.587302,0.0,0.0,0.0,1.0,False
19263,35,3,3.619048,0.0,1.0,0.0,0.0,False
...,...,...,...,...,...,...,...,...
2312,25,9,8.644444,0.0,0.0,0.0,1.0,False
16552,31,1,23.777778,1.0,0.0,0.0,0.0,False
22914,27,3,7.111111,0.0,0.0,0.0,1.0,False
23316,36,4,5.140317,1.0,0.0,0.0,0.0,False


In [ ]:
# prompt: crate an isolation forest to detect anomalies in teh result_df_encoded and fina the index of anomalies in teh data and crate a data frame like test_df_with_anomalies , use the xtrain and xtest etc using the xtrain and xtest indices created in the previuos code, and for isolation forest i think we need y test and train so creat theam , if we drop premium rate then thas sdropiing the ioput

import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split

# Assuming 'result_df_encoded' is your DataFrame from the preceding code

# Prepare the data for Isolation Forest
X = result_df_encoded.drop('Premium_rate', axis=1).values  # Assuming 'Premium_rate' is the target variable

# Split the data into training and test sets (using the same indices as before)
X_train = X[train_indices]
X_test = X[test_indices]
y_train = result_df_encoded.loc[train_indices, 'Premium_rate'].values
y_test = result_df_encoded.loc[test_indices, 'Premium_rate'].values

# Create an Isolation Forest model
model = IsolationForest(contamination=0.05, random_state=42)  # Adjust contamination as needed

# Fit the model to the training data
model.fit(X_train)

# Predict anomalies on the test data
y_pred = model.predict(X_test)

# Identify anomalies (-1 indicates an anomaly)
anomalies = y_pred == -1

# Create a copy of the original test DataFrame
test_df_with_anomalies_if = result_df_encoded.iloc[test_indices].copy()

# Add an 'anomaly' column to the test DataFrame
test_df_with_anomalies_if['anomaly'] = anomalies

# Print the results
print(f"Number of anomalies in the test set (Isolation Forest): {np.sum(anomalies)}")

# Display the test DataFrame with anomaly tags
test_df_with_anomalies_if


Number of anomalies in the test set (Isolation Forest): 402


,Age,Duration,Premium_rate,level_2_Female_NP2,level_2_Female_NS2,level_2_Male_NP2,level_2_Male_NS2,anomaly
42657,19,3,3.182540,1.0,0.0,0.0,0.0,False
38367,30,5,4.298786,0.0,0.0,1.0,0.0,False
17831,40,3,8.730159,0.0,0.0,1.0,0.0,False
7236,45,3,10.587302,0.0,0.0,0.0,1.0,False
19263,35,3,3.619048,0.0,1.0,0.0,0.0,False
...,...,...,...,...,...,...,...,...
2312,25,9,8.644444,0.0,0.0,0.0,1.0,False
16552,31,1,23.777778,1.0,0.0,0.0,0.0,False
22914,27,3,7.111111,0.0,0.0,0.0,1.0,False
23316,36,4,5.140317,1.0,0.0,0.0,0.0,False


In [10]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Assuming 'result_df_encoded' is your preprocessed DataFrame

# Extract features and target variable
X = result_df_encoded.drop('Premium_rate', axis=1).values
y = result_df_encoded['Premium_rate'].values

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Reshape input to be 3D [samples, timesteps, features]
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Define the RNN model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1, verbose=1)

# Evaluate the model
loss = model.evaluate(X_test, y_test, verbose=0)
print('Test Loss:', loss)

Epoch 1/50
974/974 [==============================] - 4s 3ms/step - loss: 355.6823 - val_loss: 188.3966
Epoch 2/50
974/974 [==============================] - 3s 3ms/step - loss: 179.6798 - val_loss: 155.8045
Epoch 3/50
974/974 [==============================] - 4s 4ms/step - loss: 155.2700 - val_loss: 136.7597
Epoch 4/50
974/974 [==============================] - 4s 4ms/step - loss: 135.9666 - val_loss: 121.6090
Epoch 5/50
974/974 [==============================] - 2s 2ms/step - loss: 124.1660 - val_loss: 113.3538
Epoch 6/50
974/974 [==============================] - 3s 3ms/step - loss: 117.3357 - val_loss: 105.6809
Epoch 7/50
974/974 [==============================] - 2s 3ms/step - loss: 111.8908 - val_loss: 103.1532
Epoch 8/50
974/974 [==============================] - 4s 4ms/step - loss: 108.0604 - val_loss: 100.2747
Epoch 9/50
974/974 [==============================] - 4s 4ms/step - loss: 104.7037 - val_loss: 96.7494
Epoch 10/50
974/974 [==============================] - 3s 3ms/ste

In [20]:
y_pred = model.predict(X_test)

# Invert scaling to get actual predicted premium values
X_test_actual = scaler.inverse_transform(X_test.reshape(X_test.shape[0], -1))

y_pred

271/271 [==============================] - 1s 4ms/step


array([[ 1.1442513],
       [ 6.4610224],
       [ 6.622149 ],
       ...,
       [ 5.744295 ],
       [ 5.683167 ],
       [13.885942 ]], dtype=float32)

In [23]:
comparison_df = pd.DataFrame(X_test_actual, columns=result_df_encoded.drop('Premium_rate', axis=1).columns)
comparison_df['Predicted_Premium_rate'] = y_pred
comparison_df['Actual_Premium_rate'] = y_test

comparison_df

,Age,Duration,level_2_Female_NP2,level_2_Female_NS2,level_2_Male_NP2,level_2_Male_NS2,Predicted_Premium_rate,Actual_Premium_rate
0,19.0,3.0,1.0,0.0,0.0,0.0,1.144251,3.182540
1,30.0,5.0,0.0,0.0,1.0,0.0,6.461022,4.298786
2,40.0,3.0,0.0,0.0,1.0,0.0,6.622149,8.730159
3,45.0,3.0,0.0,0.0,0.0,1.0,10.502310,10.587302
4,35.0,3.0,0.0,1.0,0.0,0.0,4.678481,3.619048
...,...,...,...,...,...,...,...,...
8652,25.0,9.0,0.0,0.0,0.0,1.0,8.331341,8.644444
8653,31.0,1.0,1.0,0.0,0.0,0.0,1.096175,23.777778
8654,27.0,3.0,0.0,0.0,0.0,1.0,5.744295,7.111111
8655,36.0,4.0,1.0,0.0,0.0,0.0,5.683167,5.140317
